In [1]:
import os
from dotenv import load_dotenv
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import time

load_dotenv()

True

In [ ]:
#vehicle count data
url = "https://data.cityofnewyork.us/resource/7ym2-wayt.json"
#      https://data.cityofnewyork.us/resource/btm5-ppia.json
limit = 100000
max_rows = 1712605
offset = 0
traffic_df = pd.DataFrame()
nyc_token = os.getenv("TRAFFIC_KEY")

In [ ]:
while offset < max_rows:
    this_url = url + f"?$limit={limit}&$offset={offset}&$order=:id&$$app_token={nyc_token}"
    print(this_url)
    response = requests.get(this_url)
    if response.status_code == 200:
        json_data = response.json()
        temp_df = pd.DataFrame(json_data)

        print(f"Temp Length {len(temp_df)}")

        print(temp_df.columns)

        #drop pre 2020 data 
        temp_df['yr'] = temp_df['yr'].astype(int)
        temp_df = temp_df[temp_df['yr'] >= 2020]

        #filter only manhatan
        temp_df = temp_df[temp_df['boro'].str.lower() == 'manhattan']

        print(f"Temp Length after cutoff {len(temp_df)}")
        traffic_df = pd.concat([traffic_df, temp_df])
        
    else:
        print(f"Error: {response.status_code}")
    offset = offset + limit
    print(f"Total len {len(traffic_df)}")
    time.sleep(1)


https://data.ny.gov/resource/wujg-7c2s.json?$limit=100&$offset=0&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100
Index(['transit_timestamp', 'transit_mode', 'station_complex_id',
       'station_complex', 'borough', 'payment_method', 'fare_class_category',
       'ridership', 'transfers', 'latitude', 'longitude', 'georeference'],
      dtype='object')
Temp Length after cutoff 100
Total len 100
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100&$offset=100&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100
Index(['transit_timestamp', 'transit_mode', 'station_complex_id',
       'station_complex', 'borough', 'payment_method', 'fare_class_category',
       'ridership', 'transfers', 'latitude', 'longitude', 'georeference'],
      dtype='object')
Temp Length after cutoff 100
Total len 200
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100&$offset=200&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100
Index(['transit_timestamp', 'transit_m

In [ ]:
len(traffic_df)
#313934

55609

In [ ]:
traffic_df.head()

,requestid,boro,yr,m,d,hh,mm,vol,segmentid,wktgeom,street,fromst,tost,direction
1121,32440,Manhattan,2020,11,8,20,45,62,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB
1122,32440,Manhattan,2020,11,8,21,0,63,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB
1123,32440,Manhattan,2020,11,8,21,15,43,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB
1124,32440,Manhattan,2020,11,8,21,30,46,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB
1125,32440,Manhattan,2020,11,8,21,45,36,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB


In [53]:
traffic_df['date'] = traffic_df['yr'].astype(str) + "-" + traffic_df['m'].astype(str) + "-" + traffic_df['d'].astype(str)
#df['new_col'] = df['col1'].astype(str) + '-' + df['col2'].astype(str) + '-' + df['col3'].astype(str)
traffic_df['date'] = pd.to_datetime(traffic_df['date'])
traffic_df.head()

,requestid,boro,yr,m,d,hh,mm,vol,segmentid,wktgeom,street,fromst,tost,direction,date
1121,32440,Manhattan,2020,11,8,20,45,62,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB,2020-11-08
1122,32440,Manhattan,2020,11,8,21,0,63,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB,2020-11-08
1123,32440,Manhattan,2020,11,8,21,15,43,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB,2020-11-08
1124,32440,Manhattan,2020,11,8,21,30,46,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB,2020-11-08
1125,32440,Manhattan,2020,11,8,21,45,36,72596,POINT (1004612.5006665062 254120.7358176789),DYCKMAN STREET,Sherman Avenue,Vermilyea Avenue,NB,2020-11-08


In [54]:
traffic_df['vol'] = traffic_df['vol'].astype(int)

traffic_df = traffic_df.groupby('date', as_index=False)['vol'].sum()


In [60]:
traffic_df.sort_values(by='date').to_csv('traffic.csv')